## Week 9. Deep Forward Network

기계학습 9주차는 두가지 목표로 구성된다.  
1. Tensorflow를 사용하여 목적에 맞는 모델을 생성할 수 있다.  
2. Tensorflow를 사용하여 Deep Forward Network를 학습할 수 있다.  

## 0. Load data

Dataset covertype이며 인공위성이 찍은 사진을 전처리하여 table data로 작성한 dataset이다.  
데이터를 불러오면 45000여개의 instance가 54개의 feature를 갖고 있는 것을 확인할 수 있다.  
그리고 class는 총 3개를 갖고 있다.  

In [1]:
import numpy as np
import pandas as pd

x = pd.read_csv('09_DFN_x_train.csv')
y = pd.read_csv('09_DFN_y_train.csv')

print(x.shape, y.shape)
print(y.iloc[:,0].unique())

(35527, 54) (35527, 1)
[1 2 0]


In [6]:
y

,54
0,1
1,2
2,1
3,2
4,2
...,...
35522,2
35523,0
35524,2
35525,0


## 1. Tensorflow

Tensorflow는 인공신경망을 학습시키기 위해 설계된 Library이다.  
인경신경망은 Graph구조에 기반을 두었고, node와 edge를 사용하는 다양한 연산들을 수행하기 위해 고안되었다.  
보다 빠른 연산을 위해 cpu version외에도 gpu version을 지원한다.  
수업시간에는 cpu를사용하겠다.  
만약 gpu version을 적절하게 설치했다면 아래 명령어로 GPU사용여부를 확인할수 있다.  

In [2]:
import tensorflow as tf

tf.config.get_visible_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""   ## gpu 사용을 중지함
tf.config.get_visible_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

기본적으로 Tensorflow의 구조는 MLP와 같다.  
perceptron을 계층화하여 층 하나를 쌓은 단위를 layer라고 부른다.  
layer의 unit갯수를 조절하여 계층화된 perceptron의 층 별 갯수를 조절가능하다.  
아래처럼 dense layer를 통과할 입력은 keras input으로 사용가능하다.  

In [4]:
input_layer = tf.keras.Input(3)
dense_layer = tf.keras.layers.Dense(units=4)(input_layer)

Multi layer perceptron은 계층마다 비선형 output을 부여하여 특징추출을 가능하게 한다.  
Dense layer의 output을 그대로 다음 계층의 dense layer로 전달한다면 특징 추출이 불가능하다.  
따라서 dense layer의 output을 activation function에 통과시켜 비선형성을 부여하겠다.  (ReLU)   
우리는 가장 기본적인 activation function sigmoid를 사용하겠다.  

In [5]:
activation_fn = tf.keras.activations.sigmoid(dense_layer)

위의 과정을 이어 구조를 생성하고 구조를 확인하겠다.  

In [6]:
dense_layer = tf.keras.layers.Dense(units=4)(activation_fn)
activation_fn = tf.keras.activations.sigmoid(dense_layer)
output_layer = tf.keras.layers.Dense(units=1)(activation_fn)

In [7]:
import pydot
import graphviz

model = tf.keras.models.Model(input_layer, output_layer)

tf.keras.utils.plot_model(model, show_shapes=True, expand_nested=True)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [8]:
model = tf.keras.Sequential()

model.add(tf.keras.Input(3))
model.add(tf.keras.layers.Dense(units=4, activation='sigmoid'))
model.add(tf.keras.layers.Dense(units=4, activation='sigmoid'))
model.add(tf.keras.layers.Dense(units=1))

tf.keras.utils.plot_model(model, show_shapes=True, expand_nested=True)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


둘중 편한것을 사용하는 걸 추천하고 실습에서는 functional을 사용한다 .

## 1.02 Architecture - Batch normalization & drop out

Neural neys은 error를 반복적으로 개선하는만큼 overfit 성향이 강한 모델이다.  
Overfit을 방지하기 위해 다양한 방법들이 있다.  
batchnormalization은 각 층의 출력값을 평균 0, 표준편차를 1로 만들어준다.  
drop out은 무작위로 edge를 0으로 만들어 일부 edge에 모델이 편향되는 현상을 방지한다.  
다음은 두가지 방법을 적용한 모델이다.  

In [9]:
model = tf.keras.Sequential()

model.add(tf.keras.Input(3))
model.add(tf.keras.layers.Dense(units=4))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('sigmoid'))
model.add(tf.keras.layers.Dropout(.3))
model.add(tf.keras.layers.Dense(units=4))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('sigmoid'))
model.add(tf.keras.layers.Dropout(.3))
model.add(tf.keras.layers.Dense(units=1))

tf.keras.utils.plot_model(model, show_shapes=True, expand_nested=True)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


## 1.03 Architecture - Output node & Loss function

Loss function은 architecture와 긴밀한 연관성이 있다.  
회귀문제의 경우 mean squared error, mean absolurer error를 사용하여 학습한다.  
Architecture의 output이 linear이기 때문에 두 손실 함수로 충분히 학습할 수 있다.  
차원을 하나만 사용한 분류의 경우 numerical class를 예측하는 만큼 회귀처럼 학습할 수 있다.  

## 2. Learning deep neural network

이번엔 보다 깊은 모델을 생성하게다.  
반복문을 사용하여 간단하게 5layer의 network를 생성하겠다.  
layer마다 50개의 perceptron으로 구성하겠다.  

In [10]:
def build_model():
    
    input_layer = tf.keras.Input(x.shape[1])
    
    for n in range(3):
        if n==0:
            output = input_layer
        else:
            output = drop_layer
            
        dense_layer = tf.keras.layers.Dense(units=30)(output)
        batch_layer = tf.keras.layers.BatchNormalization()(dense_layer)
        activation_fn = tf.keras.activations.tanh(batch_layer)
        drop_layer = tf.keras.layers.Dropout(.3)(activation_fn)
        
    output_layer = tf.keras.layers.Dense(units=3, activation='softmax')(drop_layer)
    
    return tf.keras.models.Model(input_layer, output_layer)

model = build_model()
model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 54)]              0         
_________________________________________________________________
dense_9 (Dense)              (None, 30)                1650      
_________________________________________________________________
batch_normalization_2 (Batch (None, 30)                120       
_________________________________________________________________
tf_op_layer_Tanh (TensorFlow [(None, 30)]              0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 30)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 30)                930       
_________________________________________________________________
batch_normalization_3 (Batch (None, 30)               

맨처음에 input할때 54+1(bias)해줘서 55*50 = 2750이 나오는것이다.  
Tensorflow model은 학습하기 전 compile을 실행한다.  
Model.compile로 실행할 수 있으며 적절한 argument를 사용하여 모델을 컴파일링하겠다.  
옵티마이저와 loss는 반드시 체워줘야하고 metrics는 필수는 아니다.

In [16]:
model.compile(optimizer='sgd',loss = 'binary_crossentropy',metrics='accuracy')

Neural network의 장점은 dataset size의 영향을 덜 받으면서 일반화할 수 있다는 점이다.  
과도한 capacity의 neural net을 사용하더라도 충분히 잘 학습한다.  

In [18]:
x = pd.read_csv('09_DFN_x_train.csv')
y = pd.read_csv('09_DFN_y_train.csv')

y = tf.keras.utils.to_categorical(y) # 원핫인코딩을 적용함
history=model.fit(x,y,
                 batch_size=200,epochs=10)

Epoch 1/10
178/178 [==============================] - 1s 4ms/step - loss: 0.6320 - accuracy: 0.4130
Epoch 2/10
178/178 [==============================] - 1s 4ms/step - loss: 0.6294 - accuracy: 0.4172
Epoch 3/10
178/178 [==============================] - 1s 4ms/step - loss: 0.6300 - accuracy: 0.4116
Epoch 4/10
178/178 [==============================] - 1s 4ms/step - loss: 0.6303 - accuracy: 0.4124
Epoch 5/10
178/178 [==============================] - 1s 4ms/step - loss: 0.6295 - accuracy: 0.4157
Epoch 6/10
178/178 [==============================] - 1s 4ms/step - loss: 0.6283 - accuracy: 0.4170
Epoch 7/10
178/178 [==============================] - 1s 4ms/step - loss: 0.6267 - accuracy: 0.4199
Epoch 8/10
178/178 [==============================] - 1s 4ms/step - loss: 0.6262 - accuracy: 0.4167
Epoch 9/10
178/178 [==============================] - 1s 4ms/step - loss: 0.6267 - accuracy: 0.4172
Epoch 10/10
178/178 [==============================] - 1s 4ms/step - loss: 0.6257 - accuracy: 0.4195

iterative learning의 특징에 따라 학습이 진행될수록 training set은 잘 설명할 수 있다.  
하지만 test set을 잘분류하는것이 중요한 만큼 training set을 적당히 학습해야한다.  
이번에는 validation set을 준비해 학습하겠다.  
실습에서는 train test split을 사용하지만 과제에는 k-fold를 사용해라  

In [24]:
from sklearn.model_selection import train_test_split
x = pd.read_csv('09_DFN_x_train.csv')
y = pd.read_csv('09_DFN_y_train.csv')

x_train, x_val, y_train, y_val = train_test_split(x,y,stratify=y)

model = build_model()
model.compile('sgd',loss = 'binary_crossentropy', metrics='accuracy')
history=model.fit(x_train, tf.keras.utils.to_categorical(y_train),
                 batch_size=200, epochs=1000,
                 validation_data=(x_val,tf.keras.utils.to_categorical(y_val)),
                 callbacks=[
                     tf.keras.callbacks.EarlyStopping(patience=10, mode='min')  # 지난 no iter랑 똑같은 기능
                 ]
                 )


Epoch 1/1000
134/134 [==============================] - 1s 6ms/step - loss: 0.6591 - accuracy: 0.4015 - val_loss: 0.6144 - val_accuracy: 0.4330
Epoch 2/1000
134/134 [==============================] - 1s 5ms/step - loss: 0.6479 - accuracy: 0.3985 - val_loss: 0.6140 - val_accuracy: 0.4330
Epoch 3/1000
134/134 [==============================] - 1s 4ms/step - loss: 0.6454 - accuracy: 0.4057 - val_loss: 0.6137 - val_accuracy: 0.4330
Epoch 4/1000
134/134 [==============================] - 1s 4ms/step - loss: 0.6424 - accuracy: 0.4086 - val_loss: 0.6134 - val_accuracy: 0.4330
Epoch 5/1000
134/134 [==============================] - 1s 4ms/step - loss: 0.6429 - accuracy: 0.4039 - val_loss: 0.6130 - val_accuracy: 0.4330
Epoch 6/1000
134/134 [==============================] - 1s 4ms/step - loss: 0.6424 - accuracy: 0.4024 - val_loss: 0.6127 - val_accuracy: 0.4330
Epoch 7/1000
134/134 [==============================] - 1s 4ms/step - loss: 0.6408 - accuracy: 0.4053 - val_loss: 0.6124 - val_accuracy:

KeyboardInterrupt: 

In [21]:
from plotly.subplots import make_subplots
from plotly import graph_objs as go
from plotly.offline import iplot
from plotly import express as px

In [23]:
fig=make_subplots(1,2,shared_xaxes=True)

name='loss'
fig.add_trace(go.Scatter(y=history.history[name],mode='lines',name=name),1,1)

name='val_loss'
fig.add_trace(go.Scatter(y=history.history[name],mode='lines',name=name),1,1)

name='accuracy'
fig.add_trace(go.Scatter(y=history.history[name],mode='lines',name=name),1,2)

name='val_accuracy'
fig.add_trace(go.Scatter(y=history.history[name],mode='lines',name=name),1,2)

iplot(fig,show_link=True)
# 학습안해서 그림 안나옴

KeyError: 'val_loss'

In [25]:
np.unique(model.predict(x_train).argmax(1))

array([2], dtype=int64)

## 3. Optimizer

Optimizer는 손실함수가 계산한 차이를 줄이는 방향으로 weights의 update를 지시하는 역활이다.  
잔차를 단순반영하는 SGD부터 시작해 더 빠르게 차이를 개선하는 momentum, RMSProp, Adam등이 있다.  
momentum과 RMSProp을 사용해보고 SGD와 비교하겠다.  

In [ ]:
model_mom=build_model()
model_mom.compile(tf.keras.optimizers.SGD(momentum=1),loss = 'binary_crossentropy',metrics='accuracy')
history=model.fit(x_train, tf.keras.utils.to_categorical(y_train),
                 batch_size=200, epochs=1000,
                 validation_data=(x_val,tf.keras.utils.to_categorical(y_val)),
                 callbacks=[
                     tf.keras.callbacks.EarlyStopping(patience=20, mode='min')  # 지난 no iter랑 똑같은 기능
                 ]
                 )


In [ ]:
model_rmsp=build_model()
model_rmsp.compile('rmsprop',loss = 'binary_crossentropy',metrics='accuracy')
history=model.fit(x_train, tf.keras.utils.to_categorical(y_train),
                 batch_size=200, epochs=1000,
                 validation_data=(x_val,tf.keras.utils.to_categorical(y_val)),
                 callbacks=[
                     tf.keras.callbacks.EarlyStopping(patience=10, mode='min')  # 지난 no iter랑 똑같은 기능
                 ]
                 )


In [ ]:
colors = px.colors.qualitative.Vivid

cols=['loss','val_loss','accuracy','val_accuracy']
fig=make_subplots(2,2,shared_xaxes=True,column_titles=['Loss','Accuracy'],row_titles=['Training','Validation'])

for n,col in enumerate(cols):
    if n==0:
        showleg=True
    else:
        showleg=False
    fig.add_trace(go.Scatter(y=history.history[col][:50],
                            mode='lines',name='SGD',legendgroup='SGD'.
                            showlegend=showleg,
                            marker=dict(color=colors[0])),n%2+1,n\\2+1)
    fig.add_trace(go.Scatter(y=history.history[col][:50],
                            mode='lines',name='Momentum',legendgroup='Momentum'.
                            showlegend=showleg,
                            marker=dict(color=colors[1])),n%2+1,n\\2+1)
    fig.add_trace(go.Scatter(y=history.history[col][:50],
                            mode='lines',name='RMSProp',legendgroup='RMSProp'.
                            showlegend=showleg,
                            marker=dict(color=colors[3])),n%2+1,n\\2+1)
iplot(fig,show_link=True)

## 4. Conclusion

영상 보기 